<a href="https://colab.research.google.com/github/IvanGoyena/Sarkany/blob/main/Guardado_Proveedor/Informe_Guardado_DEV_A_PROV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import files

In [38]:
uploaded = files.upload()

Saving Stock.xlsx to Stock (1).xlsx


In [39]:
uploaded = files.upload()

Saving Ubicaciones.xlsx to Ubicaciones (1).xlsx


In [40]:
uploaded = files.upload()

Saving 1.4 MAESTRO_ARTICULOS_LOGISTICA - COMPLETO.xlsx to 1.4 MAESTRO_ARTICULOS_LOGISTICA - COMPLETO (1).xlsx


In [41]:
# 1. Cargar archivos
Maestro = pd.read_excel('1.4 MAESTRO_ARTICULOS_LOGISTICA - COMPLETO.xlsx',skiprows=6)
Ubicaciones = pd.read_excel('Ubicaciones.xlsx')
Stock = pd.read_excel('Stock.xlsx')

In [42]:
import pandas as pd
## 1) TRANSFORMACIONES TABLA STOCK


# Lista de columnas a eliminar
columnas_a_eliminar = [
    'Pasillo', 'Zona', 'Orden de salida', 'Proveedor',
    'Estado', 'Carro', 'Paquete', 'Y', 'X', 'Particiones'
]

# Eliminar las columnas del DataFrame
Stock = Stock.drop(columns=columnas_a_eliminar)



## 2) TRANSFORMACIONES TABLA MAESTRO

# Lista de columnas a eliminar
columnas_a_eliminar = [
    'CODIGO', 'CODIGO_MODELO', 'MODELO + COLOR', 'CODIGO_DESCRIPCION',
    'TEMPORADA', 'RUBRO', 'SUBRUBRO', 'TEMPORADA_ORIGEN', 'COLOR',
    'NOMBRE_FANTASIA', 'ESTILO', 'FAMILIA', 'GRUPO',
    'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
    'TALLE','MARCA'
]

# Eliminar las columnas del DataFrame
Maestro = Maestro.drop(columns=columnas_a_eliminar, errors='ignore')
Maestro = Maestro.dropna(subset=['PROVEEDOR'])


In [43]:
## EXTRACCION DE DATOS DE OTRAS TABLAS

# Extraer solo la columna clave y PROVEEDOR
maestro_subset = Maestro[['CODIGO_BARRAS', 'PROVEEDOR']]

# Realizar el merge
Stock = Stock.merge(maestro_subset, left_on='Artículo', right_on='CODIGO_BARRAS', how='left')

# Eliminar la columna duplicada de clave que vino del merge
Stock = Stock.drop(columns=['CODIGO_BARRAS'])

# Si existe una columna duplicada de proveedor, eliminala
if 'PROVEEDOR_x' in Stock.columns and 'PROVEEDOR_y' in Stock.columns:
    Stock = Stock.drop(columns=['PROVEEDOR_x'])
    Stock = Stock.rename(columns={'PROVEEDOR_y': 'PROVEEDOR'})
elif 'PROVEEDOR_y' in Stock.columns:
    Stock = Stock.rename(columns={'PROVEEDOR_y': 'PROVEEDOR'})


    # Extraer solo columnas necesarias de Ubicaciones
ubicaciones_subset = Ubicaciones[['Razon Social', 'Ubicación']]

# Hacer el merge usando la columna PROVEEDOR como clave
Stock = Stock.merge(ubicaciones_subset, left_on='PROVEEDOR', right_on='Razon Social', how='left')

# Drop the redundant 'Razon Social' column after merge
Stock = Stock.drop(columns=['Razon Social'])


# If there's a duplicate 'Ubicacion' column, rename the new one
if 'Ubicacion_x' in Stock.columns and 'Ubicación_y' in Stock.columns:
    Stock = Stock.drop(columns=['Ubicacion_x'])
    Stock = Stock.rename(columns={'Ubicación_y': 'Ubicacion'})
elif 'Ubicación_y' in Stock.columns:
    Stock = Stock.rename(columns={'Ubicación_y': 'Ubicacion'})

In [44]:
Stock = Stock.sort_values(by='Ubicacion')
Stock.head()

,Artículo,Cantidad,Ubicación_x,Contenedor,Estado de usuario,PROVEEDOR,Ubicacion
61,1S5ICKANI%CQ36,1,STG_LI,168373,ARREGLOS,ESGOLUSE S.A.,PA-08-A-01-01
22,1S5IGFILE%AR36,1,STG_LI,168373,ARREGLOS,ESGOLUSE S.A.,PA-08-A-01-01
19,1S5ISDISSA%NG37,1,STG_LI,168373,ARREGLOS,ESGOLUSE S.A.,PA-08-A-01-01
17,1S5IGFILE%AR38,1,STG_LI,168373,ARREGLOS,ESGOLUSE S.A.,PA-08-A-01-01
16,1S5ITBALD%NG37,2,STG_LI,168373,ARREGLOS,ESGOLUSE S.A.,PA-08-A-01-01


In [45]:
Stock.to_csv('Informe_Guardado_DEV_A_PROV.csv', index=False, encoding='utf-8')
